In [4]:
import os
import sys
import topogenesis as tg
import pyvista as pv
import pandas as pd
import trimesh as tm
from itertools import cycle
import numpy as np
import pickle
import networkx as nx
import numpy.ma as ma
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
from ladybug.sunpath import Sunpath

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('voxelized_envelope_6m_voxel_size.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

In [10]:
s = tg.create_stencil("von_neumann", 1, 1)
s

stencil([[[0, 0, 0],
          [0, 1, 0],
          [0, 0, 0]],

         [[0, 1, 0],
          [1, 1, 1],
          [0, 1, 0]],

         [[0, 0, 0],
          [0, 1, 0],
          [0, 0, 0]]])

In [22]:
env_lat_flat = envelope_lattice.flatten()

adj_matrix = np.zeros((env_lat_flat.size,env_lat_flat.size), dtype=int)

all_vox_neighs_inds = envelope_lattice.find_neighbours(s)
all_vox_neighs = all_vox_neighs_inds[:, 1:]
all_vox_inds = np.indices(all_vox_neighs.shape)[0]

adj_matrix[all_vox_inds.flatten(), all_vox_neighs.flatten()] = 1

adj_matrix[env_lat_flat==0, :] *= 0
adj_matrix[:, env_lat_flat==0] *= 0

g = nx.from_numpy_array(adj_matrix)

In [26]:
g.remove_nodes_from(list(nx.isolates(g)))

In [31]:
g_cc = nx.algorithms.connected_components(g)

buildingid_lat_flat = env_lat_flat * 0 - 1

for i, nodes in enumerate(g_cc):
    buildingid_lat_flat[list(nodes)] = i

In [33]:
buildingid_lat = tg.to_lattice(buildingid_lat_flat.reshape(envelope_lattice.shape), envelope_lattice)

In [35]:
# initiating the plotter
p = pv.Plotter(notebook=True)

base_lat = envelope_lattice

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = base_lat.shape
# The bottom left corner of the data set
grid.origin = base_lat.minbound
# These are the cell sizes along each axis
grid.spacing = base_lat.unit

# Add the data values to the cell data
grid.point_arrays["Distance from South Facade"] = buildingid_lat.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
# envelope_lattice.fast_vis(p)

threshold = grid.threshold([-0.1, buildingid_lat_flat.max() + 0.1])
p.add_mesh(threshold, opacity=1.0)
# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(141.41095176547094, 312.41095176547094, 279.41095176547094),
 (-108.0, 63.0, 30.0),
 (0.0, 0.0, 1.0)]

In [60]:
build_dist_y_lat = envelope_lattice * 0.0
for i in range(buildingid_lat.max()+1):
    dim = 0
    building_1_inds = np.argwhere(buildingid_lat==i)

    max_bb = building_1_inds.max(axis=0)
    min_bb = building_1_inds.min(axis=0)
    build_1_y_dist = max_bb[dim] - building_1_inds[:, dim]

    build_dist_y_lat[np.where(buildingid_lat==i)] = build_1_y_dist + 1
    print(i)
build_dist_y_lat /= build_dist_y_lat.max()

0
1
2
3


In [61]:
# initiating the plotter
p = pv.Plotter(notebook=True)

base_lat = build_dist_y_lat

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = base_lat.shape
# The bottom left corner of the data set
grid.origin = base_lat.minbound
# These are the cell sizes along each axis
grid.spacing = base_lat.unit

# Add the data values to the cell data
grid.point_arrays["Distance from South Facade"] = base_lat.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
# envelope_lattice.fast_vis(p)

threshold = grid.threshold([0.01, 1])
p.add_mesh(threshold, opacity=1.0)
# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(141.41095176547094, 312.41095176547094, 279.41095176547094),
 (-108.0, 63.0, 30.0),
 (0.0, 0.0, 1.0)]